In [3]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [5]:
pth = r'\\169.254.138.20\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376'  # path of ndpi files

pth_mat = r'\\169.254.138.20\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376\1x_python\cropped\matfiles'  # path .mat files with crop data (got from matlab file)

In [6]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]

# Get a list of NDPI images and their base names (without extensions)
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]

# Find the matching files based on their base names
matching_files = set(mat_files).intersection(ndpi_files)

WSIs = [filename + '.ndpi' for filename in matching_files]


FileNotFoundError: [WinError 53] The network path was not found: '\\\\169.254.138.20\\Andre\\data\\Ashleigh fallopian tube\\fallopian tubes\\AJER376\\1x_python\\cropped\\matfiles'

In [4]:
print(WSIs)

['fallopian_tubes_AJER376_117.ndpi', 'fallopian_tubes_AJER376_331.ndpi', 'fallopian_tubes_AJER376_217.ndpi', 'fallopian_tubes_AJER376_439.ndpi', 'fallopian_tubes_AJER376_523.ndpi']


In [5]:
res20x=0.4416
res1x=8

ratio = res1x/res20x
# print(ratio)
# ratio= 16
outpth = os.path.join(pth,'stardist_training_tiles512')
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [8]:
from scipy.io import loadmat
import h5py

file_format = '.tif'

for i, file in enumerate(mat_files):
    mat_file_name = os.path.join(pth_mat, file)
    # print(mat_file_name)
    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,WSIs[i][:-5]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(mat_files)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(mat_files)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]
        crop_width = 512
        crop_height = 512
        #data = loadmat(mat_file_name)
        try:
            data = loadmat(mat_file_name+'.mat')

            crop_x = int(data['crop_x']*ratio)
            crop_y = int(data['crop_y']*ratio)
        except:
            print(mat_file_name+'.mat')
            data = h5py.File(mat_file_name+'.mat', 'r')

            crop_x = int(data['crop_x'][()]*ratio)
            crop_y = int(data['crop_y'][()]*ratio)
        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)
        
        # print(data['crop_x'])
        

        lower_x = crop_x
        upper_y = crop_y
        # upper_x = crop_x + crop_width
        # lower_y = crop_y - crop_height

        print(crop_x)
        print(crop_y)
        # print(crop_width)
        # print(crop_height)
        
        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width,crop_height)) # level = 0 in middle
        # region = WSI_slide.read_region((crop_y, crop_x), 0, (crop_width, crop_height)) # level = 0 in middle

        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Skipping cropping for fallopian_tubes_AJER376_117.ndpi (1/5)... Already exists
Cropping fallopian_tubes_AJER376_331.ndpi (2/5)
\\169.254.138.20\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376\1x_python\cropped\matfiles\fallopian_tubes_AJER376_217.mat
18962
19941
Cropping fallopian_tubes_AJER376_217.ndpi (3/5)
\\169.254.138.20\Andre\data\Ashleigh fallopian tube\fallopian tubes\AJER376\1x_python\cropped\matfiles\fallopian_tubes_AJER376_331.mat
26843
21734


C:\Users\Andre\AppData\Local\Temp\ipykernel_17716\3669700110.py:37: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  crop_x = int(data['crop_x'][()]*ratio)
C:\Users\Andre\AppData\Local\Temp\ipykernel_17716\3669700110.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  crop_y = int(data['crop_y'][()]*ratio)


Skipping cropping for fallopian_tubes_AJER376_439.ndpi (4/5)... Already exists
Skipping cropping for fallopian_tubes_AJER376_523.ndpi (5/5)... Already exists
